# Extraindo Dados
___

Esse arquivo tem o objetivo de extrair dados do dataframe BSI e criar uma tabela que contenha os seguintes dados:

- o identificador do discente; 
- o sexo do discente;
- o ano em que o discente ingressou na instituição;
- o ano e período da última matrícula realizada pelo discente;
- total de sementres cursados pelo discente;
- um calcúlo de semestres cursados dividido pelo o número de semestre ideal do curso;
- a carga horária cumprida pelo discente;
- a carga horária cumprida pelo discente dividida pela carga horaria ideal;
- tempo relativo que o discente evadiu ou concluiu o curso;
- status atual do discente;
- número total de faltas de cada discente na disciplina que ele cursou;
- nota de cada discente nas disciplinas que ele cursou;
- média final de cada discente nas disciplinas que ele cursou;
- quantidade de vezes que o discente se matriculou nas disciplinas;
- quantidade de vezes que o discente se matriculou em cada disciplina;
- quantidade de vezes que o discente foi aprovado em cada disciplina;
- quantidade de vezes que o discente foi reprovado em cada disciplina.

Importando o Pandas e o Csv.

In [1383]:
import pandas as pd
import csv

Leitura do arquivo em csv e carregamento desses dados em um DataFrame usando o ponto e vírgula como separador.

In [1384]:
df_dados = pd.read_csv('dataframe-bsi-2009-2022.csv', sep=';')

In [1385]:
pd.set_option('display.max_columns', 130)

# Tratamento de Dados

Listando as colunas do dataframe.

In [1386]:
df_dados.columns

Index(['discente', 'unidade', 'nota', 'media_final', 'frequencia', 'descricao',
       'ano', 'id_componente', 'nome', 'ch_total', 'sexo', 'ano_nascimento',
       'ano_ingresso', 'status'],
      dtype='object')

Transformando a coluna *media_final* no tipo float.

In [1387]:
df_dados['media_final']=df_dados['media_final'].apply(lambda x: str(x).replace(',' , '.'))
df_dados['media_final'] = df_dados['media_final'].astype('float64')

Transformando a coluna *nota* no tipo float.

In [1388]:
df_dados['nota']=df_dados['nota'].apply(lambda x: str(x).replace(',' , '.'))
df_dados['nota'] = df_dados['nota'].astype('float64')

Informações sobre o tipo de cada coluna do dataframe.

In [1389]:
df_dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54584 entries, 0 to 54583
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   discente        54584 non-null  object 
 1   unidade         50972 non-null  float64
 2   nota            48321 non-null  float64
 3   media_final     47687 non-null  float64
 4   frequencia      50471 non-null  float64
 5   descricao       54584 non-null  object 
 6   ano             54584 non-null  int64  
 7   id_componente   54584 non-null  int64  
 8   nome            54584 non-null  object 
 9   ch_total        54584 non-null  int64  
 10  sexo            54584 non-null  int64  
 11  ano_nascimento  54584 non-null  int64  
 12  ano_ingresso    54584 non-null  int64  
 13  status          54584 non-null  object 
dtypes: float64(4), int64(6), object(4)
memory usage: 5.8+ MB


Listando os dados NaN.

In [1390]:
df_dados.isnull().sum()

discente             0
unidade           3612
nota              6263
media_final       6897
frequencia        4113
descricao            0
ano                  0
id_componente        0
nome                 0
ch_total             0
sexo                 0
ano_nascimento       0
ano_ingresso         0
status               0
dtype: int64

Preenchendo os dados NaN da coluna *unidade* com 1.

In [1391]:
df_dados.loc[:, 'unidade'] = df_dados['unidade'].fillna(1)

Apagando da coluna *descricao* a condição *INDEFERIDO* e *EXCLUIDA*

In [1392]:
df_dados = df_dados[~df_dados['descricao'].isin(['INDEFERIDO', 'EXCLUIDA'])]

Verificar os valores faltosos na coluna *nota* com base na condição da coluna *descricao*.

In [1393]:
condicao_cancelado = df_dados['descricao'] == 'CANCELADO'
df_dados.loc[condicao_cancelado, 'nota'] = -2

condicao_trancado = df_dados['descricao'] == 'TRANCADO'
df_dados.loc[condicao_trancado, 'nota'] = -4

condicao_desistencia = df_dados['descricao'] == 'DESISTENCIA'
df_dados.loc[condicao_desistencia, 'nota'] = -7

Verificar os valores faltosos na coluna *media_final* com base na condição da coluna *descricao*.

In [1394]:
condicao_cancelado = df_dados['descricao'] == 'CANCELADO'
df_dados.loc[condicao_cancelado, 'media_final'] = -2

condicao_trancado = df_dados['descricao'] == 'TRANCADO'
df_dados.loc[condicao_trancado, 'media_final'] = -4

condicao_desistencia = df_dados['descricao'] == 'DESISTENCIA'
df_dados.loc[condicao_desistencia, 'media_final'] = -7

Verificar os valores faltosos na coluna *frequencia* com base na condição da coluna *descricao*.

In [1395]:
condicao_cancelado = df_dados['descricao'] == 'CANCELADO'
df_dados.loc[condicao_cancelado, 'frequencia'] = -2

condicao_trancado = df_dados['descricao'] == 'TRANCADO'
df_dados.loc[condicao_trancado, 'frequencia'] = -4

condicao_desistencia = df_dados['descricao'] == 'DESISTENCIA'
df_dados.loc[condicao_desistencia, 'frequencia'] = -7

Contando os valores dos dados da coluna *descricao*.

In [1396]:
df_dados.descricao.value_counts()

descricao
APROVADO                            28307
APROVADO POR NOTA                    8178
REPROVADO                            5790
REPROVADO POR MÉDIA E POR FALTAS     3804
CANCELADO                            2730
TRANCADO                             1913
DESISTENCIA                           457
REPROVADO POR FALTAS                  138
REPROVADO POR NOTA                     99
REPROVADO POR NOTA E FALTA             42
Name: count, dtype: int64

Criar uma função para mapear a classificação da coluna *descricao*.

In [1397]:
def mapear_classificacao(descricao):
    mapeamento = {
        'APROVADO': 2,
        'APROVADO POR NOTA': 1,
        'REPROVADO': -1,
        'REPROVADO POR FALTAS': -1,
        'REPROVADO POR NOTA': -1,
        'REPROVADO POR MÉDIA E POR FALTAS': -2,
        'REPROVADO POR NOTA E FALTA': -2,
        'CANCELADO': -3,
        'EXCLUIDA': -3,
        'TRANCADO': -3,
        'DESISTENCIA': -4,
    }
    return mapeamento.get(descricao, None)

Aplicar a função de mapeamento e criar a coluna *classificacao*.

In [1398]:
df_dados.loc[:, 'classificacao'] = df_dados['descricao'].map(mapear_classificacao)

Fazendo um recorte da nossa análise, vamos começar por disciplinas obrigatórias do Bacharelado em Sistemas da Informação (BSI):

In [1399]:
lista_obrigatórias = [
                'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO',
                'INTRODUÇÃO À INFORMÁTICA',
                'FUNDAMENTOS DE MATEMÁTICA',
                'LÓGICA',
                'TEORIA GERAL DA ADMINISTRAÇÃO',
                'PROGRAMAÇÃO',
                'CÁLCULO DIFERENCIAL E INTEGRAL',
                'TEORIA GERAL DOS SISTEMAS',
                'PROGRAMAÇÃO ORIENTADA A OBJETOS I',
                'ESTRUTURA DE DADOS',
                'ÁLGEBRA LINEAR',
                'ORGANIZAÇÃO, SISTEMAS E MÉTODOS',
                'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO',
                'PROGRAMAÇÃO WEB',
                'ARQUITETURA DE COMPUTADORES',
                'PROBABILIDADE E ESTATÍSTICA',
                'BANCO DE DADOS',
                'ENGENHARIA DE SOFTWARE I',
                'PROGRAMAÇÃO ORIENTADA A OBJETOS II',
                'SISTEMAS OPERACIONAIS',
                'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS',
                'ENGENHARIA DE SOFTWARE II',
                'REDES DE COMPUTADORES',
                'EMPREENDEDORISMO EM INFORMÁTICA',
                'GESTÃO DE PROJETO DE SOFTWARE',
                'PROGRAMAÇÃO VISUAL',
                'MATEMÁTICA FINANCEIRA',
                'SISTEMAS DE APOIO À DECISÃO',
                'ÉTICA',
                ]
condição_nome = f"nome in {lista_obrigatórias}"
df_dados_filtrado = df_dados.query(condição_nome)

## Extraindo dados

In [1400]:
df_dados_filtrado.columns

Index(['discente', 'unidade', 'nota', 'media_final', 'frequencia', 'descricao',
       'ano', 'id_componente', 'nome', 'ch_total', 'sexo', 'ano_nascimento',
       'ano_ingresso', 'status', 'classificacao'],
      dtype='object')

### Colunas com as **Disciplinas**

Filtrar os dados onde a coluna **unidade** foi preenchida com 1.

In [1401]:
filtrando_unidade = df_dados_filtrado[df_dados_filtrado['unidade'] == 1]

Calculando a quantidade de vezes que cada discente cursou cada *disciplina*.

In [1402]:
quantidade_disciplinas = filtrando_unidade.groupby(['discente', 'nome']).size().reset_index(name='quantidade')


Pivotando as *disciplinas*.

In [1403]:
tabela_final = quantidade_disciplinas.pivot(index='discente', columns='nome', values='quantidade').reset_index()

### Coluna com o **Ano de Ingresso**

Adicionando o *ano_ingresso* para cada discente.

In [1404]:
ano_ingresso_discente = df_dados_filtrado.drop_duplicates(subset=['discente'])[['discente', 'ano_ingresso']]
tabela_final = tabela_final.merge(ano_ingresso_discente, on='discente', how='left')

### Coluna com o **Sexo** do Discente

Definir a opção global para permitir o downcasting silencioso

In [1405]:
pd.set_option('future.no_silent_downcasting', True)

Adicionar as colunas *mulher* e *homem* usando o método assign para evitar SettingWithCopyWarning.

In [1406]:
df_dados_filtrado = df_dados_filtrado.assign(
    mulher=df_dados_filtrado['sexo'].map({0: 0, 1: 1}),
    homem=df_dados_filtrado['sexo'].map({0: 1, 1: 0})
)

Criar DataFrames com discente e as novas colunas.

In [1407]:
sexo_discente = df_dados_filtrado.drop_duplicates(subset=['discente'])[['discente', 'mulher', 'homem']]

Mesclar com tabela_final.

In [1408]:
tabela_final = tabela_final.merge(sexo_discente, on='discente', how='left')


### Coluna com o **Status** do Discente

Alterando o conteúdo da coluna *status*.

In [1409]:
df_dados_filtrado['status'] = df_dados_filtrado['status'].replace({
    'CANCELADO': -1, 
    'ATIVO': 0, 'ATIVO - FORMANDO': 0,
    'FORMADO': 1,'CONCLUÍDO': 1})

Adicionando o *status* para cada discente.


In [1410]:
status_discente = df_dados_filtrado.drop_duplicates(subset=['discente'])[['discente', 'status']]
tabela_final = tabela_final.merge(status_discente, on='discente', how='left')
tabela_final 

,discente,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,ARQUITETURA DE COMPUTADORES,BANCO DE DADOS,CÁLCULO DIFERENCIAL E INTEGRAL,EMPREENDEDORISMO EM INFORMÁTICA,ENGENHARIA DE SOFTWARE I,ENGENHARIA DE SOFTWARE II,ESTRUTURA DE DADOS,FUNDAMENTOS DE MATEMÁTICA,FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO,GESTÃO DE PROJETO DE SOFTWARE,INTRODUÇÃO À INFORMÁTICA,LÓGICA,MATEMÁTICA FINANCEIRA,"ORGANIZAÇÃO, SISTEMAS E MÉTODOS",PROBABILIDADE E ESTATÍSTICA,PROGRAMAÇÃO,PROGRAMAÇÃO ORIENTADA A OBJETOS I,PROGRAMAÇÃO ORIENTADA A OBJETOS II,PROGRAMAÇÃO VISUAL,PROGRAMAÇÃO WEB,PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS,REDES DE COMPUTADORES,SISTEMAS DE APOIO À DECISÃO,SISTEMAS OPERACIONAIS,TEORIA GERAL DA ADMINISTRAÇÃO,TEORIA GERAL DOS SISTEMAS,ÁLGEBRA LINEAR,ÉTICA,ano_ingresso,mulher,homem,status
0,001cea3c82e2010681f2cdeab21e5ecf,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,2018,0,1,-1
1,005c14d7c07bf7980b60c703f99c5ee7,1.0,2.0,1.0,1.0,NaN,1.0,NaN,3.0,2.0,1.0,NaN,2.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,NaN,1.0,2.0,1.0,NaN,1.0,1.0,1.0,2.0,1.0,2018,0,1,-1
2,0107fd69d8cd7e3d30dede96fb68bfe5,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,2.0,2.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0,2011,0,1,-1
3,014789363f7940922e71e710ee9d22bc,2.0,3.0,1.0,NaN,1.0,1.0,1.0,2.0,3.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2016,0,1,1
4,014f0dec46fe7a9c5836527662e1df10,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,0,1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,fe802d8d85de6f842749468401d1146c,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,2022,0,1,0
676,fe87dfa176a74fc10a5cb701b9fb5dd4,1.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,3.0,1.0,1.0,1.0,3.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2016,0,1,1
677,fec9ed6026d55ecdf514c640312c3d08,1.0,1.0,1.0,1.0,NaN,1.0,NaN,1.0,1.0,1.0,NaN,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,NaN,2.0,1.0,NaN,NaN,NaN,1.0,1.0,1.0,NaN,2020,0,1,0
678,ff56f2c5048dae0797fd3e851572b80c,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,4.0,1.0,1.0,4.0,4.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,2014,0,1,1


### Coluna com o **classificacao** da Descrição

Adicionando o *classificacao* para cada discente.

In [1411]:
classificacao_descricao = df_dados_filtrado.drop_duplicates(subset=['discente'])[['discente', 'classificacao']]
tabela_final = tabela_final.merge(classificacao_descricao, on='discente', how='left')

### Coluna com a **Carga Horária Cumprida**

In [1412]:
filtrando_unidade = df_dados_filtrado[df_dados_filtrado['unidade'] == 1]

Agrupando por discente e somando a *carga horária*.

In [1413]:
df_carga_horaria_cumprida = filtrando_unidade.groupby('discente')['ch_total'].sum().reset_index()

Adicionando a *carga horária cumprida* de cada discente e renomeando a coluna.

In [1414]:
df_carga_horaria_cumprida.rename(columns={'ch_total': 'ch_cumprida'}, inplace=True)
tabela_final = tabela_final.merge(df_carga_horaria_cumprida, on='discente', how='left')

Dividindo a *carga horária cumprida* do discente pela carga horária das disciplinas obrigatórias.

In [1415]:
tabela_final['ch_cumprida_dividida'] = tabela_final['ch_cumprida'] / 1830

### Coluna com a **Carga Horária Cumprida** com **Reprovação**

Filtrando dados pela unidade.

In [1416]:
filtrando_unidade = df_dados_filtrado[df_dados_filtrado['unidade'] == 1]

Filtrar as disciplinas *reprovadas*.

In [1417]:
reprovadas_status = [
    'REPROVADO',
    'REPROVADO POR NOTA',
    'REPROVADO POR FALTAS',
    'REPROVADO POR NOTA E FALTA',
    'REPROVADO POR MÉDIA E POR FALTAS'
]

In [1418]:
df_reprovadas = filtrando_unidade[filtrando_unidade['descricao'].isin(reprovadas_status)]

Agrupar por discente e somar a *carga horária das disciplinas reprovadas*.

In [1419]:
ch_reprovacao = df_reprovadas.groupby('discente')['ch_total'].sum().reset_index()

Renomear a coluna resultante.

In [1420]:
ch_reprovacao.columns = ['discente', 'ch_reprovacao']

Adicionando a *carga horária de reprovação* de cada discente.

In [1421]:
tabela_final = tabela_final.merge(ch_reprovacao, on='discente', how='left')
df_dados_filtrado = df_dados_filtrado.merge(ch_reprovacao, on='discente', how='left')
df_dados_filtrado

,discente,unidade,nota,media_final,frequencia,descricao,ano,id_componente,nome,ch_total,sexo,ano_nascimento,ano_ingresso,status,classificacao,mulher,homem,ch_reprovacao
0,bad4e0ec61e635f382e7e32c32d5ad11,1.0,0.0,0.0,44.0,REPROVADO POR FALTAS,20091,2037000,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,90,1,1991,2009,-1,-1,1,0,330.0
1,bad4e0ec61e635f382e7e32c32d5ad11,2.0,0.0,0.0,44.0,REPROVADO POR FALTAS,20091,2037000,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,90,1,1991,2009,-1,-1,1,0,330.0
2,bad4e0ec61e635f382e7e32c32d5ad11,3.0,0.0,0.0,44.0,REPROVADO POR FALTAS,20091,2037000,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,90,1,1991,2009,-1,-1,1,0,330.0
3,bee926721a5b23a098fdfb850bf8d5b5,1.0,0.0,0.0,52.0,REPROVADO POR FALTAS,20091,2037000,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,90,0,1983,2009,-1,-1,0,1,330.0
4,bee926721a5b23a098fdfb850bf8d5b5,2.0,0.0,0.0,52.0,REPROVADO POR FALTAS,20091,2037000,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,90,0,1983,2009,-1,-1,0,1,330.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38085,22f4aed4a073c5e9515a8669e9c102f3,2.0,6.9,5.2,0.0,APROVADO POR NOTA,20222,62764,PROGRAMAÇÃO VISUAL,60,0,1984,2019,0,1,0,1,240.0
38086,22f4aed4a073c5e9515a8669e9c102f3,3.0,5.0,5.2,0.0,APROVADO POR NOTA,20222,62764,PROGRAMAÇÃO VISUAL,60,0,1984,2019,0,1,0,1,240.0
38087,943463d2de8c6a60ec5f5d959ba7f1ac,1.0,10.0,10.0,0.0,APROVADO,20222,2054400,MATEMÁTICA FINANCEIRA,60,0,1999,2018,1,2,0,1,60.0
38088,943463d2de8c6a60ec5f5d959ba7f1ac,2.0,10.0,10.0,0.0,APROVADO,20222,2054400,MATEMÁTICA FINANCEIRA,60,0,1999,2018,1,2,0,1,60.0


Substituindo NaN por 0 nas colunas.

In [1422]:
tabela_final = tabela_final.fillna(0)

### Coluna com **taxa de reprovação**

In [1423]:
tabela_final['taxa_reprovacao'] = tabela_final['ch_reprovacao'] / tabela_final['ch_cumprida'] 

### Coluna com o **Semestre**

Função para calcular o *semestre* com base no *ano de ingresso* e o *ano cursado.*

In [1424]:
def calcular_semestre(ano_ingresso, ano_cursado):
    try:
        # Convertendo os anos para strings e ajustando para o formato correto
        ano_ingresso_str = str(ano_ingresso) + '1' if len(str(ano_ingresso)) == 4 else str(ano_ingresso)
        ano_cursado_str = str(ano_cursado) + '1' if len(str(ano_cursado)) == 4 else str(ano_cursado)
        
        ano_ingresso_num = int(ano_ingresso_str[:4])
        semestre_ingresso = int(ano_ingresso_str[4])
        
        ano_cursado_num = int(ano_cursado_str[:4])
        semestre_cursado = int(ano_cursado_str[4])
        
        anos_passados = (ano_cursado_num - ano_ingresso_num) * 2
        semestres_passados = anos_passados + (semestre_cursado - semestre_ingresso)
        
        return semestres_passados + 1  # +1 porque o semestre de ingresso é contado como o primeiro
    except (IndexError, ValueError):
        return pd.NA  # Retorna NA se houver um problema com o formato dos anos

Corrigir os valores das colunas *ano_ingresso* e *ano*.

In [1425]:
df_dados_filtrado['ano_ingresso'] = df_dados_filtrado['ano_ingresso'].astype(str)
df_dados_filtrado['ano'] = df_dados_filtrado['ano'].astype(str)

Aplicar a função para criar a coluna *semestre*.

In [1426]:
df_dados_filtrado['semestre'] = df_dados_filtrado.apply(
    lambda x: calcular_semestre(x['ano_ingresso'], x['ano']), axis=1
)

In [1427]:
# Agrupar por discente e disciplina e coletar os semestres
semestres_por_disciplina = df_dados_filtrado.groupby(['discente', 'nome'])['semestre'].apply(lambda x: ', '.join(map(str, x.unique()))).reset_index()

# Renomear a coluna para algo mais descritivo
semestres_por_disciplina.rename(columns={'semestre': 'semestres_cursados'}, inplace=True)

# Exibir o resultado
semestres_por_disciplina

,discente,nome,semestres_cursados
0,001cea3c82e2010681f2cdeab21e5ecf,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,1
1,001cea3c82e2010681f2cdeab21e5ecf,FUNDAMENTOS DE MATEMÁTICA,1
2,001cea3c82e2010681f2cdeab21e5ecf,INTRODUÇÃO À INFORMÁTICA,1
3,001cea3c82e2010681f2cdeab21e5ecf,LÓGICA,1
4,001cea3c82e2010681f2cdeab21e5ecf,TEORIA GERAL DA ADMINISTRAÇÃO,1
...,...,...,...
10701,ff56f2c5048dae0797fd3e851572b80c,TEORIA GERAL DOS SISTEMAS,2
10702,ff56f2c5048dae0797fd3e851572b80c,ÁLGEBRA LINEAR,8
10703,ffe447b2fe2058d45d5f756349a26f45,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,1
10704,ffe447b2fe2058d45d5f756349a26f45,INTRODUÇÃO À INFORMÁTICA,1


In [1428]:
# Supondo que tabela_final já existe e tem as colunas 'discente' e 'disciplina'
df_dados_filtrado = df_dados_filtrado.merge(semestres_por_disciplina, on=['discente' , 'nome'], how='left')
df_dados_filtrado

,discente,unidade,nota,media_final,frequencia,descricao,ano,id_componente,nome,ch_total,sexo,ano_nascimento,ano_ingresso,status,classificacao,mulher,homem,ch_reprovacao,semestre,semestres_cursados
0,bad4e0ec61e635f382e7e32c32d5ad11,1.0,0.0,0.0,44.0,REPROVADO POR FALTAS,20091,2037000,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,90,1,1991,2009,-1,-1,1,0,330.0,1,1
1,bad4e0ec61e635f382e7e32c32d5ad11,2.0,0.0,0.0,44.0,REPROVADO POR FALTAS,20091,2037000,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,90,1,1991,2009,-1,-1,1,0,330.0,1,1
2,bad4e0ec61e635f382e7e32c32d5ad11,3.0,0.0,0.0,44.0,REPROVADO POR FALTAS,20091,2037000,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,90,1,1991,2009,-1,-1,1,0,330.0,1,1
3,bee926721a5b23a098fdfb850bf8d5b5,1.0,0.0,0.0,52.0,REPROVADO POR FALTAS,20091,2037000,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,90,0,1983,2009,-1,-1,0,1,330.0,1,1
4,bee926721a5b23a098fdfb850bf8d5b5,2.0,0.0,0.0,52.0,REPROVADO POR FALTAS,20091,2037000,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,90,0,1983,2009,-1,-1,0,1,330.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38085,22f4aed4a073c5e9515a8669e9c102f3,2.0,6.9,5.2,0.0,APROVADO POR NOTA,20222,62764,PROGRAMAÇÃO VISUAL,60,0,1984,2019,0,1,0,1,240.0,8,"7, 8"
38086,22f4aed4a073c5e9515a8669e9c102f3,3.0,5.0,5.2,0.0,APROVADO POR NOTA,20222,62764,PROGRAMAÇÃO VISUAL,60,0,1984,2019,0,1,0,1,240.0,8,"7, 8"
38087,943463d2de8c6a60ec5f5d959ba7f1ac,1.0,10.0,10.0,0.0,APROVADO,20222,2054400,MATEMÁTICA FINANCEIRA,60,0,1999,2018,1,2,0,1,60.0,10,10
38088,943463d2de8c6a60ec5f5d959ba7f1ac,2.0,10.0,10.0,0.0,APROVADO,20222,2054400,MATEMÁTICA FINANCEIRA,60,0,1999,2018,1,2,0,1,60.0,10,10


In [1429]:
semestres = df_dados_filtrado[['discente', 'semestre']].drop_duplicates("discente")
tabela_final = tabela_final.merge(semestres, on='discente', how='left')
tabela_final


,discente,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,ARQUITETURA DE COMPUTADORES,BANCO DE DADOS,CÁLCULO DIFERENCIAL E INTEGRAL,EMPREENDEDORISMO EM INFORMÁTICA,ENGENHARIA DE SOFTWARE I,ENGENHARIA DE SOFTWARE II,ESTRUTURA DE DADOS,FUNDAMENTOS DE MATEMÁTICA,FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO,GESTÃO DE PROJETO DE SOFTWARE,INTRODUÇÃO À INFORMÁTICA,LÓGICA,MATEMÁTICA FINANCEIRA,"ORGANIZAÇÃO, SISTEMAS E MÉTODOS",PROBABILIDADE E ESTATÍSTICA,PROGRAMAÇÃO,PROGRAMAÇÃO ORIENTADA A OBJETOS I,PROGRAMAÇÃO ORIENTADA A OBJETOS II,PROGRAMAÇÃO VISUAL,PROGRAMAÇÃO WEB,PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS,REDES DE COMPUTADORES,SISTEMAS DE APOIO À DECISÃO,SISTEMAS OPERACIONAIS,TEORIA GERAL DA ADMINISTRAÇÃO,TEORIA GERAL DOS SISTEMAS,ÁLGEBRA LINEAR,ÉTICA,ano_ingresso,mulher,homem,status,classificacao,ch_cumprida,ch_cumprida_dividida,ch_reprovacao,taxa_reprovacao,semestre
0,001cea3c82e2010681f2cdeab21e5ecf,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2018,0,1,-1,-4,330,0.180328,0.0,0.000000,1
1,005c14d7c07bf7980b60c703f99c5ee7,1.0,2.0,1.0,1.0,0.0,1.0,0.0,3.0,2.0,1.0,0.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,1.0,2.0,1.0,0.0,1.0,1.0,1.0,2.0,1.0,2018,0,1,-1,-2,2160,1.180328,960.0,0.444444,1
2,0107fd69d8cd7e3d30dede96fb68bfe5,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,2011,0,1,-1,-1,870,0.475410,270.0,0.310345,1
3,014789363f7940922e71e710ee9d22bc,2.0,3.0,1.0,0.0,1.0,1.0,1.0,2.0,3.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2016,0,1,1,-1,2310,1.262295,540.0,0.233766,1
4,014f0dec46fe7a9c5836527662e1df10,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020,0,1,-1,-3,240,0.131148,0.0,0.000000,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,fe802d8d85de6f842749468401d1146c,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2022,0,1,0,-1,540,0.295082,270.0,0.500000,1
676,fe87dfa176a74fc10a5cb701b9fb5dd4,1.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,3.0,1.0,1.0,1.0,3.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2016,0,1,1,2,2430,1.327869,480.0,0.197531,1
677,fec9ed6026d55ecdf514c640312c3d08,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,2.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,2020,0,1,0,2,1350,0.737705,0.0,0.000000,2
678,ff56f2c5048dae0797fd3e851572b80c,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,4.0,1.0,1.0,4.0,4.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,2014,0,1,1,-1,2670,1.459016,630.0,0.235955,1


In [1430]:
df_dados_filtrado.columns

Index(['discente', 'unidade', 'nota', 'media_final', 'frequencia', 'descricao',
       'ano', 'id_componente', 'nome', 'ch_total', 'sexo', 'ano_nascimento',
       'ano_ingresso', 'status', 'classificacao', 'mulher', 'homem',
       'ch_reprovacao', 'semestre', 'semestres_cursados'],
      dtype='object')

### Coluna com o Total de **Semestre Cursado**

Identificar *semestres* únicos cursados por cada discente.

In [1431]:
semestres_unicos_por_discente = df_dados_filtrado.groupby('discente')['ano'].nunique().reset_index()
semestres_unicos_por_discente.rename(columns={'ano' : 'semestre_total'}, inplace=True)

Adicionando a quantidade de *semestres* cursados por cada discente.

In [1432]:
tabela_final = tabela_final.merge(semestres_unicos_por_discente, on='discente', how='left')

In [1433]:
df_dados_filtrado =df_dados_filtrado.merge(semestres_unicos_por_discente, on='discente', how='left')

Dividindo a quantidade de *semestres cursados* pela quantidade de *semestres estimados* no curso.

In [1434]:
tabela_final['semestre_dividido'] = tabela_final['semestre_total'] / 8

### Coluna com o **Último Período** Matrículado

Encontrar o *último período* que cada aluno estudou no curso.

In [1435]:
ultimo_periodo = df_dados_filtrado.groupby('discente')['ano'].max().reset_index()
ultimo_periodo.rename(columns={'ano': 'ultimo_periodo'}, inplace=True)

Adicionando as informações do *último período* de cada discente.

In [1436]:
tabela_final = tabela_final.merge(ultimo_periodo, on='discente', how='left')

### Coluna com a quantidade de vezes que o discente se **Matriculou**

Soma dos valores das colunas para contabilizar a *matrícula* por discente.

In [1437]:
matricula =      tabela_final['ALGORITMOS E LÓGICA DE PROGRAMAÇÃO'] + \
                 tabela_final['INTRODUÇÃO À INFORMÁTICA'] + \
                 tabela_final['FUNDAMENTOS DE MATEMÁTICA'] + \
                 tabela_final['LÓGICA'] + \
                 tabela_final['TEORIA GERAL DA ADMINISTRAÇÃO'] + \
                 tabela_final['PROGRAMAÇÃO'] + \
                 tabela_final['CÁLCULO DIFERENCIAL E INTEGRAL'] + \
                 tabela_final['TEORIA GERAL DOS SISTEMAS'] + \
                 tabela_final['PROGRAMAÇÃO ORIENTADA A OBJETOS I'] + \
                 tabela_final['ESTRUTURA DE DADOS'] + \
                 tabela_final['ÁLGEBRA LINEAR'] + \
                 tabela_final['ORGANIZAÇÃO, SISTEMAS E MÉTODOS'] + \
                 tabela_final['FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO'] + \
                 tabela_final['PROGRAMAÇÃO WEB'] + \
                 tabela_final['ARQUITETURA DE COMPUTADORES'] + \
                 tabela_final['PROBABILIDADE E ESTATÍSTICA'] + \
                 tabela_final['BANCO DE DADOS'] + \
                 tabela_final['ENGENHARIA DE SOFTWARE I'] + \
                 tabela_final['PROGRAMAÇÃO ORIENTADA A OBJETOS II'] + \
                 tabela_final['SISTEMAS OPERACIONAIS'] + \
                 tabela_final['PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS'] + \
                 tabela_final['ENGENHARIA DE SOFTWARE II'] + \
                 tabela_final['REDES DE COMPUTADORES'] + \
                 tabela_final['EMPREENDEDORISMO EM INFORMÁTICA'] + \
                 tabela_final['GESTÃO DE PROJETO DE SOFTWARE'] + \
                 tabela_final['PROGRAMAÇÃO VISUAL'] + \
                 tabela_final['MATEMÁTICA FINANCEIRA'] + \
                 tabela_final['SISTEMAS DE APOIO À DECISÃO'] + \
                 tabela_final['ÉTICA']

Adicionando a coluna *matrícula* na tabela.

In [1438]:
tabela_final['matricula'] = matricula

### Coluna com o **Tempo Relativo** que o discente passou no curso

Converter a coluna *ch_cumprida_dividida* para tipo numérico.

In [1439]:
tabela_final['semestre_dividido'] = pd.to_numeric(tabela_final['semestre_dividido'], errors='coerce')

Converter a coluna *status* para tipo numérico.

In [1440]:
tabela_final['status'] = pd.to_numeric(tabela_final['status'], errors='coerce')

Multiplicar os valores da coluna *ch_cumprida_dividida* pela coluna *status*.

In [1441]:
tabela_final['tempo_relativo'] = tabela_final['semestre_dividido'] * tabela_final['status']

### Coluna com a quantidade de vezes que o discente foi **Aprovado**

Filtrando dados pela unidade.

In [1442]:
filtrando_unidade = df_dados_filtrado[df_dados_filtrado['unidade'] == 1]

Listar todas as disciplinas.

In [1443]:
disciplinas = df_dados_filtrado['nome'].unique()

Criar uma tabela de *aprovações*.

In [1444]:
aprovacoes = filtrando_unidade[filtrando_unidade['descricao'].isin([
    'APROVADO',
    'APROVADO POR NOTA',
])]

Agrupar por discente e nome da disciplina para contar as *aprovações*.

In [1445]:
aprovacoes_count = aprovacoes.groupby(['discente', 'nome']).size().unstack(fill_value=0)

Renomear as colunas para incluir *_APROVADO*.

In [1446]:
aprovacoes_count.columns = [f"{disciplina}_APROVADO" for disciplina in aprovacoes_count.columns]

Mesclar as *aprovações* de volta ao DataFrame original.

In [1447]:
# Criar `df_final` removendo duplicatas em 'discente' e juntando com `aprovacoes_count`
df_final = df_dados_filtrado.drop_duplicates('discente').set_index('discente').join(aprovacoes_count, on='discente').fillna(0).reset_index()

# Selecionar as colunas de df_final que não estão presentes em tabela_final
colunas_para_juntar = [col for col in df_final.columns if col not in tabela_final.columns or col == 'discente']

# Selecionar apenas as colunas necessárias
df_final_rename = df_final[colunas_para_juntar]

# Realizar a junção com tabela_final
tabela_final = tabela_final.merge(df_final_rename, on='discente', how='left')

### Coluna com a quantidade de vezes que o discente foi **Reprovado**

Listar todas as disciplinas.

In [1448]:
disciplinas = df_dados_filtrado['nome'].unique()

Criar uma tabela de *reprovações* com o filtro da 1 unidade.

In [1449]:
reprovacoes = filtrando_unidade[filtrando_unidade['descricao'].isin([
    'REPROVADO',
    'REPROVADO POR NOTA',
    'REPROVADO POR FALTAS',
    'REPROVADO POR NOTA E FALTA',
    'REPROVADO POR MÉDIA E POR FALTAS'
])]

Agrupar por discente e nome da disciplina para contar as *reprovações*.

In [1450]:
reprovacoes_count = reprovacoes.groupby(['discente', 'nome']).size().unstack(fill_value=0)

Renomear as colunas para incluir *_REPROVADO*.

In [1451]:
reprovacoes_count.columns = [f"{disciplina}_REPROVADO" for disciplina in reprovacoes_count.columns]

Mesclar as *reprovações* de volta ao DataFrame original.

In [1452]:
# Criar `df_final` removendo duplicatas em 'discente' e juntando com `reprovacoes_count`
df_final = df_dados_filtrado.drop_duplicates('discente').set_index('discente').join(reprovacoes_count, on='discente').fillna(0).reset_index()

# Selecionar as colunas de df_final que não estão presentes em tabela_final
colunas_para_juntar = [col for col in df_final.columns if col not in tabela_final.columns or col == 'discente']

# Selecionar apenas as colunas necessárias
df_final_rename = df_final[colunas_para_juntar]

# Realizar a junção com tabela_final
tabela_final = tabela_final.merge(df_final_rename, on='discente', how='left')

In [1453]:
tabela_final.columns

Index(['discente', 'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO',
       'ARQUITETURA DE COMPUTADORES', 'BANCO DE DADOS',
       'CÁLCULO DIFERENCIAL E INTEGRAL', 'EMPREENDEDORISMO EM INFORMÁTICA',
       'ENGENHARIA DE SOFTWARE I', 'ENGENHARIA DE SOFTWARE II',
       'ESTRUTURA DE DADOS', 'FUNDAMENTOS DE MATEMÁTICA',
       ...
       'PROGRAMAÇÃO VISUAL_REPROVADO', 'PROGRAMAÇÃO WEB_REPROVADO',
       'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_REPROVADO',
       'REDES DE COMPUTADORES_REPROVADO',
       'SISTEMAS DE APOIO À DECISÃO_REPROVADO',
       'SISTEMAS OPERACIONAIS_REPROVADO',
       'TEORIA GERAL DA ADMINISTRAÇÃO_REPROVADO',
       'TEORIA GERAL DOS SISTEMAS_REPROVADO', 'ÁLGEBRA LINEAR_REPROVADO',
       'ÉTICA_REPROVADO'],
      dtype='object', length=115)

In [1454]:
df_dados_filtrado.columns

Index(['discente', 'unidade', 'nota', 'media_final', 'frequencia', 'descricao',
       'ano', 'id_componente', 'nome', 'ch_total', 'sexo', 'ano_nascimento',
       'ano_ingresso', 'status', 'classificacao', 'mulher', 'homem',
       'ch_reprovacao', 'semestre', 'semestres_cursados', 'semestre_total'],
      dtype='object')

## Pivoteamento

In [1455]:
df_pivot_f = df_dados_filtrado.pivot_table(index='discente', columns='nome', values='frequencia')
df_pivot_f = df_pivot_f.add_suffix('_FALTAS')


In [1456]:

df_pivot_nota = df_dados_filtrado.pivot_table(index='discente', columns= ['nome','unidade'], values='nota', aggfunc='first')
df_pivot_nota.columns = df_pivot_nota.columns.set_levels(
    [level + '_NOTA_' for level in df_pivot_nota.columns.levels[0]],  # Aplicando o sufixo aos nomes do nível 'nome'
    level=0  # Indicando que estamos modificando o primeiro nível (nome)
)

In [1457]:
# Flatten the columns by joining different levels
df_pivot_nota.columns = [''.join(map(str, col)).strip() for col in df_pivot_nota.columns.values]
df_pivot_nota

,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_NOTA_1.0,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_NOTA_2.0,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_NOTA_3.0,ARQUITETURA DE COMPUTADORES_NOTA_1.0,ARQUITETURA DE COMPUTADORES_NOTA_2.0,ARQUITETURA DE COMPUTADORES_NOTA_3.0,BANCO DE DADOS_NOTA_1.0,BANCO DE DADOS_NOTA_2.0,BANCO DE DADOS_NOTA_3.0,CÁLCULO DIFERENCIAL E INTEGRAL_NOTA_1.0,CÁLCULO DIFERENCIAL E INTEGRAL_NOTA_2.0,CÁLCULO DIFERENCIAL E INTEGRAL_NOTA_3.0,EMPREENDEDORISMO EM INFORMÁTICA_NOTA_1.0,EMPREENDEDORISMO EM INFORMÁTICA_NOTA_2.0,EMPREENDEDORISMO EM INFORMÁTICA_NOTA_3.0,ENGENHARIA DE SOFTWARE I_NOTA_1.0,ENGENHARIA DE SOFTWARE I_NOTA_2.0,ENGENHARIA DE SOFTWARE I_NOTA_3.0,ENGENHARIA DE SOFTWARE II_NOTA_1.0,ENGENHARIA DE SOFTWARE II_NOTA_2.0,ENGENHARIA DE SOFTWARE II_NOTA_3.0,ESTRUTURA DE DADOS_NOTA_1.0,ESTRUTURA DE DADOS_NOTA_2.0,ESTRUTURA DE DADOS_NOTA_3.0,FUNDAMENTOS DE MATEMÁTICA_NOTA_1.0,FUNDAMENTOS DE MATEMÁTICA_NOTA_2.0,FUNDAMENTOS DE MATEMÁTICA_NOTA_3.0,FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_NOTA_1.0,FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_NOTA_2.0,FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_NOTA_3.0,GESTÃO DE PROJETO DE SOFTWARE_NOTA_1.0,GESTÃO DE PROJETO DE SOFTWARE_NOTA_2.0,GESTÃO DE PROJETO DE SOFTWARE_NOTA_3.0,INTRODUÇÃO À INFORMÁTICA_NOTA_1.0,INTRODUÇÃO À INFORMÁTICA_NOTA_2.0,INTRODUÇÃO À INFORMÁTICA_NOTA_3.0,LÓGICA_NOTA_1.0,LÓGICA_NOTA_2.0,LÓGICA_NOTA_3.0,MATEMÁTICA FINANCEIRA_NOTA_1.0,MATEMÁTICA FINANCEIRA_NOTA_2.0,MATEMÁTICA FINANCEIRA_NOTA_3.0,"ORGANIZAÇÃO, SISTEMAS E MÉTODOS_NOTA_1.0","ORGANIZAÇÃO, SISTEMAS E MÉTODOS_NOTA_2.0","ORGANIZAÇÃO, SISTEMAS E MÉTODOS_NOTA_3.0",PROBABILIDADE E ESTATÍSTICA_NOTA_1.0,PROBABILIDADE E ESTATÍSTICA_NOTA_2.0,PROBABILIDADE E ESTATÍSTICA_NOTA_3.0,PROGRAMAÇÃO_NOTA_1.0,PROGRAMAÇÃO_NOTA_2.0,PROGRAMAÇÃO_NOTA_3.0,PROGRAMAÇÃO ORIENTADA A OBJETOS I_NOTA_1.0,PROGRAMAÇÃO ORIENTADA A OBJETOS I_NOTA_2.0,PROGRAMAÇÃO ORIENTADA A OBJETOS I_NOTA_3.0,PROGRAMAÇÃO ORIENTADA A OBJETOS II_NOTA_1.0,PROGRAMAÇÃO ORIENTADA A OBJETOS II_NOTA_2.0,PROGRAMAÇÃO ORIENTADA A OBJETOS II_NOTA_3.0,PROGRAMAÇÃO VISUAL_NOTA_1.0,PROGRAMAÇÃO VISUAL_NOTA_2.0,PROGRAMAÇÃO VISUAL_NOTA_3.0,PROGRAMAÇÃO WEB_NOTA_1.0,PROGRAMAÇÃO WEB_NOTA_2.0,PROGRAMAÇÃO WEB_NOTA_3.0,PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_NOTA_1.0,PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_NOTA_2.0,PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_NOTA_3.0,REDES DE COMPUTADORES_NOTA_1.0,REDES DE COMPUTADORES_NOTA_2.0,REDES DE COMPUTADORES_NOTA_3.0,SISTEMAS DE APOIO À DECISÃO_NOTA_1.0,SISTEMAS DE APOIO À DECISÃO_NOTA_2.0,SISTEMAS DE APOIO À DECISÃO_NOTA_3.0,SISTEMAS OPERACIONAIS_NOTA_1.0,SISTEMAS OPERACIONAIS_NOTA_2.0,SISTEMAS OPERACIONAIS_NOTA_3.0,TEORIA GERAL DA ADMINISTRAÇÃO_NOTA_1.0,TEORIA GERAL DA ADMINISTRAÇÃO_NOTA_2.0,TEORIA GERAL DA ADMINISTRAÇÃO_NOTA_3.0,TEORIA GERAL DOS SISTEMAS_NOTA_1.0,TEORIA GERAL DOS SISTEMAS_NOTA_2.0,TEORIA GERAL DOS SISTEMAS_NOTA_3.0,ÁLGEBRA LINEAR_NOTA_1.0,ÁLGEBRA LINEAR_NOTA_2.0,ÁLGEBRA LINEAR_NOTA_3.0,ÉTICA_NOTA_1.0,ÉTICA_NOTA_2.0,ÉTICA_NOTA_3.0
discente,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
001cea3c82e2010681f2cdeab21e5ecf,-7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-7.0,NaN,NaN,-7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
005c14d7c07bf7980b60c703f99c5ee7,6.5,3.5,7.7,4.8,0.0,0.0,7.1,3.6,6.3,0.9,0.0,0.0,NaN,NaN,NaN,0.0,5.1,0.0,NaN,NaN,NaN,0.0,0.0,0.0,2.5,0.0,0.0,7.6,9.5,10.0,NaN,NaN,NaN,0.0,0.0,0.0,5.3,3.8,0.0,5.0,8.5,9.3,7.6,8.6,8.0,4.0,4.0,7.0,4.6,2.5,1.0,7.5,6.4,3.0,7.5,8.0,5.0,NaN,NaN,NaN,8.6,9.0,8.5,-4.0,-4.0,-4.0,1.0,0.0,0.0,NaN,NaN,NaN,7.3,2.7,6.8,8.5,8.7,8.9,4.9,6.0,6.5,0.0,0.0,0.0,9.5,9.0,9.0
0107fd69d8cd7e3d30dede96fb68bfe5,6.4,4.0,5.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,0.5,1.3,-2.0,-2.0,-2.0,NaN,NaN,N

In [1458]:
df_pivot_mf = df_dados_filtrado.pivot_table(index='discente', columns='nome', values='media_final',aggfunc='first')
df_pivot_mf = df_pivot_mf.add_suffix('_MEDIA1')

In [1459]:
df_pivot_mf_2 = df_dados_filtrado.pivot_table(index='discente', columns='nome', values='media_final', aggfunc='mean')
df_pivot_mf_2 = df_pivot_mf_2.add_suffix('_MEDIA2')

In [1460]:

# Mesclar df1_pivot e df2_pivot
df_merged_1 = pd.merge(df_pivot_f, df_pivot_nota, on='discente')
df_merged_2 = pd.merge(df_merged_1, df_pivot_mf, on='discente')
df_merged_3 = pd.merge(df_merged_2, df_pivot_mf_2, on='discente')
df_merged_3

,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_FALTAS,ARQUITETURA DE COMPUTADORES_FALTAS,BANCO DE DADOS_FALTAS,CÁLCULO DIFERENCIAL E INTEGRAL_FALTAS,EMPREENDEDORISMO EM INFORMÁTICA_FALTAS,ENGENHARIA DE SOFTWARE I_FALTAS,ENGENHARIA DE SOFTWARE II_FALTAS,ESTRUTURA DE DADOS_FALTAS,FUNDAMENTOS DE MATEMÁTICA_FALTAS,FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_FALTAS,GESTÃO DE PROJETO DE SOFTWARE_FALTAS,INTRODUÇÃO À INFORMÁTICA_FALTAS,LÓGICA_FALTAS,MATEMÁTICA FINANCEIRA_FALTAS,"ORGANIZAÇÃO, SISTEMAS E MÉTODOS_FALTAS",PROBABILIDADE E ESTATÍSTICA_FALTAS,PROGRAMAÇÃO_FALTAS,PROGRAMAÇÃO ORIENTADA A OBJETOS I_FALTAS,PROGRAMAÇÃO ORIENTADA A OBJETOS II_FALTAS,PROGRAMAÇÃO VISUAL_FALTAS,PROGRAMAÇÃO WEB_FALTAS,PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_FALTAS,REDES DE COMPUTADORES_FALTAS,SISTEMAS DE APOIO À DECISÃO_FALTAS,SISTEMAS OPERACIONAIS_FALTAS,TEORIA GERAL DA ADMINISTRAÇÃO_FALTAS,TEORIA GERAL DOS SISTEMAS_FALTAS,ÁLGEBRA LINEAR_FALTAS,ÉTICA_FALTAS,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_NOTA_1.0,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_NOTA_2.0,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_NOTA_3.0,ARQUITETURA DE COMPUTADORES_NOTA_1.0,ARQUITETURA DE COMPUTADORES_NOTA_2.0,ARQUITETURA DE COMPUTADORES_NOTA_3.0,BANCO DE DADOS_NOTA_1.0,BANCO DE DADOS_NOTA_2.0,BANCO DE DADOS_NOTA_3.0,CÁLCULO DIFERENCIAL E INTEGRAL_NOTA_1.0,CÁLCULO DIFERENCIAL E INTEGRAL_NOTA_2.0,CÁLCULO DIFERENCIAL E INTEGRAL_NOTA_3.0,EMPREENDEDORISMO EM INFORMÁTICA_NOTA_1.0,EMPREENDEDORISMO EM INFORMÁTICA_NOTA_2.0,EMPREENDEDORISMO EM INFORMÁTICA_NOTA_3.0,ENGENHARIA DE SOFTWARE I_NOTA_1.0,ENGENHARIA DE SOFTWARE I_NOTA_2.0,ENGENHARIA DE SOFTWARE I_NOTA_3.0,ENGENHARIA DE SOFTWARE II_NOTA_1.0,ENGENHARIA DE SOFTWARE II_NOTA_2.0,ENGENHARIA DE SOFTWARE II_NOTA_3.0,ESTRUTURA DE DADOS_NOTA_1.0,ESTRUTURA DE DADOS_NOTA_2.0,ESTRUTURA DE DADOS_NOTA_3.0,FUNDAMENTOS DE MATEMÁTICA_NOTA_1.0,FUNDAMENTOS DE MATEMÁTICA_NOTA_2.0,FUNDAMENTOS DE MATEMÁTICA_NOTA_3.0,FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_NOTA_1.0,FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_NOTA_2.0,FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_NOTA_3.0,GESTÃO DE PROJETO DE SOFTWARE_NOTA_1.0,GESTÃO DE PROJETO DE SOFTWARE_NOTA_2.0,GESTÃO DE PROJETO DE SOFTWARE_NOTA_3.0,INTRODUÇÃO À INFORMÁTICA_NOTA_1.0,INTRODUÇÃO À INFORMÁTICA_NOTA_2.0,INTRODUÇÃO À INFORMÁTICA_NOTA_3.0,...,TEORIA GERAL DOS SISTEMAS_NOTA_3.0,ÁLGEBRA LINEAR_NOTA_1.0,ÁLGEBRA LINEAR_NOTA_2.0,ÁLGEBRA LINEAR_NOTA_3.0,ÉTICA_NOTA_1.0,ÉTICA_NOTA_2.0,ÉTICA_NOTA_3.0,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_MEDIA1,ARQUITETURA DE COMPUTADORES_MEDIA1,BANCO DE DADOS_MEDIA1,CÁLCULO DIFERENCIAL E INTEGRAL_MEDIA1,EMPREENDEDORISMO EM INFORMÁTICA_MEDIA1,ENGENHARIA DE SOFTWARE I_MEDIA1,ENGENHARIA DE SOFTWARE II_MEDIA1,ESTRUTURA DE DADOS_MEDIA1,FUNDAMENTOS DE MATEMÁTICA_MEDIA1,FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_MEDIA1,GESTÃO DE PROJETO DE SOFTWARE_MEDIA1,INTRODUÇÃO À INFORMÁTICA_MEDIA1,LÓGICA_MEDIA1,MATEMÁTICA FINANCEIRA_MEDIA1,"ORGANIZAÇÃO, SISTEMAS E MÉTODOS_MEDIA1",PROBABILIDADE E ESTATÍSTICA_MEDIA1,PROGRAMAÇÃO_MEDIA1,PROGRAMAÇÃO ORIENTADA A OBJETOS I_MEDIA1,PROGRAMAÇÃO ORIENTADA A OBJETOS II_MEDIA1,PROGRAMAÇÃO VISUAL_MEDIA1,PROGRAMAÇÃO WEB_MEDIA1,PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_MEDIA1,REDES DE COMPUTADORES_MEDIA1,SISTEMAS DE APOIO À DECISÃO_MEDIA1,SISTEMAS OPERACIONAIS_MEDIA1,TEORIA GERAL DA ADMINISTRAÇÃO_MEDIA1,TEORIA GERAL DOS SISTEMAS_MEDIA1,ÁLGEBRA LINEAR_MEDIA1,ÉTICA_MEDIA1,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_MEDIA2,ARQUITETURA DE COMPUTADORES_MEDIA2,BANCO DE DADOS_MEDIA2,CÁLCULO DIFERENCIAL E INTEGRAL_MEDIA2,EMPREENDEDORISMO EM INFORMÁTICA_MEDIA2,ENGENHARIA DE SOFTWARE I_MEDIA2,ENGENHARIA DE SOFTWARE II_MEDIA2,ESTRUTURA DE DADOS_MEDIA2,FUNDAMENTOS DE MATEMÁTICA_MEDIA2,FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_MEDIA2,GESTÃO DE PROJETO DE SOFTWARE_MEDIA2,INTRODUÇÃO À INFORMÁTICA_MEDIA2,LÓGICA_MEDIA2,MATEMÁTICA FINANCEIRA_MEDIA2,"ORGANIZAÇÃO, SISTEMAS E MÉTODOS_MEDIA2",PROBABILIDADE E ESTATÍSTICA_MEDIA2,PROGRAMAÇÃO_MEDIA2,PROGRAMAÇÃO ORIENTADA A OBJETOS I_MEDIA2,PROGRAMAÇÃO ORIENTADA A OBJETOS II_MEDIA2,PROGRAMAÇÃO VISUAL_MEDIA2,PROGRA

Mudando a ordem das colunas.

Reordenando as colunas.

In [1461]:
'''tabela_final = tabela_final[colunas_ordenadas]
tabela_final'''

'tabela_final = tabela_final[colunas_ordenadas]\ntabela_final'

Para salvar o DataFrame no formato CSV com a separação por ';' e garantir que os dados numéricos estejam no formato correto.

In [1462]:
df_dados_filtrado.to_csv('df_filtrado.csv', index=False, sep=',', quoting=csv.QUOTE_NONNUMERIC)

In [1463]:
tabela_final.to_csv('tabela_final.csv', index=False, sep=',', quoting=csv.QUOTE_NONNUMERIC)